## Mapas de Ponto

Além dos dados _geométricos_ fornecidos por arquivos GeoJSON ou TopoJSON, muitos conjuntos de dados tabulares incluem informações geográficas na forma de campos de coordenadas `longitude` e `latitude`, ou referências a regiões geográficas, como nomes de países, estados, códigos postais, _etc._ Esses dados podem ser convertidos em coordenadas usando um [serviço de geocodificação](https://en.wikipedia.org/wiki/Geocoding).  

Em alguns casos, os dados de localização são tão ricos que conseguimos identificar padrões significativos apenas projetando os pontos de dados &mdash; sem a necessidade de um mapa de base!  

Vamos analisar um conjunto de dados com códigos postais de 5 dígitos dos Estados Unidos, incluindo as coordenadas `longitude` e `latitude` de cada agência dos correios, além do campo `zip_code`.


In [ ]:
zipcodes = data.zipcodes.url
zipcodes

Podemos visualizar cada agência dos correios usando uma pequena marca `square` (de 1 pixel). No entanto, para definir as posições, _não_ utilizamos os canais `x` e `y`. _Por que isso acontece?_

Embora as projeções cartográficas mapeiem coordenadas (`longitude`, `latitude`) para (`x`, `y`), esse mapeamento pode ser feito de diversas maneiras. Não há garantia, por exemplo, de que `longitude` → `x` e `latitude` → `y`!  

Em vez disso, o Altair inclui canais especiais de codificação `longitude` e `latitude` para lidar com coordenadas geográficas. Esses canais indicam quais campos de dados devem ser mapeados para `longitude` e `latitude`, e então aplicam uma projeção para converter essas coordenadas em posições (`x`, `y`) no gráfico.


In [ ]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # apply the field named 'longitude' to the longitude channel
    latitude='latitude:Q'    # apply the field named 'latitude' to the latitude channel
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Ao plotar apenas os códigos postais, conseguimos visualizar o contorno dos Estados Unidos e identificar padrões significativos na densidade das agências dos correios, sem a necessidade de um mapa de base ou elementos adicionais de referência!_  

Usamos a projeção `albersUsa`, que faz algumas adaptações na geometria real da Terra, posicionando versões redimensionadas do Alasca e do Havaí no canto inferior esquerdo. Como não especificamos os parâmetros `scale` ou `translate` da projeção, o Altair os ajusta automaticamente para encaixar os dados visualizados.  

Agora podemos explorar mais perguntas sobre nosso conjunto de dados. Por exemplo, há algum padrão na alocação dos códigos postais? Para avaliar essa questão, podemos adicionar uma codificação de cor baseada no primeiro dígito do código postal.  

Primeiro, aplicamos uma transformação `calculate` para extrair o primeiro dígito e, em seguida, utilizamos o canal de cor para representá-lo visualmente:


In [ ]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Para dar zoom um dígito específico, adicione uma transformação de filtro para limitar os dados exibidos! Experimente incluir uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar um único dígito e atualizar dinamicamente o mapa. E lembre-se de usar strings (\`'1'\`) em vez de números (\`1\`) ao filtrar valores de dígitos!_  

(Este exemplo foi inspirado na clássica visualização [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)  

Podemos também nos perguntar o que a _sequência_ dos códigos postais pode indicar. Uma maneira de explorar essa questão é conectar cada código postal consecutivo usando uma marca `line`, como na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states) de Robert Kosara:


In [ ]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Agora podemos observar como os códigos postais se agrupam ainda mais em áreas menores, indicando uma alocação hierárquica dos códigos por localização, embora com alguma variabilidade notável dentro de clusters locais._  

Se você prestou atenção nos nossos mapas anteriores, pode ter notado que alguns códigos postais estão sendo plotados no canto superior esquerdo! Eles correspondem a locais como Porto Rico ou Samoa Americana, que possuem códigos postais dos EUA, mas são mapeados para coordenadas `null` (`0`, `0`) pela projeção `albersUsa`. Além disso, Alasca e Havaí podem complicar a visualização das conexões entre os segmentos de linha.  

Para lidar com isso, o código acima inclui um filtro adicional que remove pontos fora dos intervalos escolhidos de `longitude` e `latitude`.  

_Remova o filtro acima para ver o que acontece!_
